In [1]:
#pip install langchain-community==0.2.4 langchain==0.2.3 faiss-cpu==1.8.0 unstructured==0.14.5 unstructured[pdf]==0.14.5 transformers==4.41.2 sentence-transformers==3.0.1

**Importing the Dependencies**

In [2]:
import os

from langchain_community.llms import Ollama
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredFileLoader
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain.chains import RetrievalQA
from llama_index.core import SimpleDirectoryReader
import pytesseract

In [3]:
# loading the LLM
llm = Ollama(
    model="llama3:instruct",
    temperature=0
)

In [4]:
pytesseract.pytesseract.tesseract_cmd = 'C:/Users/shobhandeb.paul/AppData/Local/Programs/Tesseract-OCR/tesseract.exe'  # your path may be different

In [5]:
# loading the document
# poppler_path = 'poppler-24.07.0/Library/bin'
# loader = SimpleDirectoryReader(input_files=[r'C:\Project\GenAI\NIPS-2017-attention-is-all-you-need-Paper.pdf']).load_data()
# documents = loader

file_path = r'C:\Project\GenAI\NIPS-2017-attention-is-all-you-need-Paper.pdf'
loader = PyPDFLoader(file_path)
document = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunked_documents = text_splitter.split_documents(document)


In [6]:
# # create document chunks
# text_splitter = CharacterTextSplitter(separator="/n",
#                                       chunk_size=1000,
#                                       chunk_overlap=200)

# text_splitter = RecursiveCharacterTextSplitter(
#     # Set a really small chunk size, just to show.
#     chunk_size=100,
#     chunk_overlap=20,
#     length_function=len,
#     is_separator_regex=False,
# )

In [7]:
# text_chunks = text_splitter.split_documents(documents)

In [8]:
# loading the vector embedding model
embeddings = HuggingFaceEmbeddings()

C:\Users\shobhandeb.paul\AppData\Local\Temp\ipykernel_19016\803604610.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings()
c:\Project\GenAI\.venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Project\GenAI\.venv\Lib\site-packages\huggingface_hub\file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `fo

In [9]:
# knowledge_base = FAISS.from_documents(text_chunks, embeddings)
knowledge_base = FAISS.from_documents(chunked_documents, embeddings)

In [10]:
# retrieval QA chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=knowledge_base.as_retriever())

In [11]:
question = "What is this document about?"
response = qa_chain.invoke({"query": question})
print(response["result"])

This document appears to be an academic paper or research article on the Transformer model, specifically discussing its architecture and training regime for machine translation tasks.


In [12]:
question = "What is the architecture discussed in the model?"
response = qa_chain.invoke({"query": question})
print(response["result"])

The architecture discussed in the model is a Transformer model that follows an overall architecture using stacked self-attention and point-wise, fully connected layers for both the encoder and decoder. The encoder is composed of a stack of 6 identical layers, each with two sub-layers: a multi-head self-attention mechanism and a simple, position-wise fully connected feed-forward network. The decoder is also composed of a stack of 6 identical layers, with an additional third sub-layer that performs multi-head attention over the output of the encoder stack.
